In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# Load the model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [ ]:
new_tensor = None
red_tensor=None
black_tensor=None

#load Image

In [ ]:
# Load and preprocess an image
image = Image.open("black.png")
inputs = processor(images=image, return_tensors="pt")

# Define a hook to capture the activations
activations = {}

def get_activation(name):
    def hook(model, input, output):
        if isinstance(output, torch.Tensor):
            activations[name] = output.detach()
        elif isinstance(output, tuple):
            activations[name] = tuple(o.detach() if isinstance(o, torch.Tensor) else o for o in output)
    return hook

# Register hooks for all layers
for name, module in model.vision_model.named_modules():
    module.register_forward_hook(get_activation(name))

# Run the image through the model
with torch.no_grad():
    model.get_image_features(**inputs)

In [ ]:
import copy

# Access the activations


for layer_name, activation in activations.items():
    if isinstance(activation, torch.Tensor):
      if(layer_name=="post_layernorm"):
        print(f"Layer: {layer_name}, Activation shape: {activation.shape}")
        activation = [f"{diff:.2f}" for diff in activation.flatten()] # Flatten the array if it's multi-dimensional
        print(activation)
        new_tensor = copy.deepcopy(activations[layer_name])

    elif isinstance(activation, tuple):
        for i, act in enumerate(activation):
            if isinstance(act, torch.Tensor):
              if(layer_name=="post_layernorm"):
                print(f"Layer: {layer_name} - Part {i}, Activation shape: {act.shape}")
                act = [f"{diff:.2f}" for diff in act.flatten()] # Flatten the array if it's multi-dimensional
                print(act)
                new_tensor = copy.deepcopy(activation[i])
            else:
              if(layer_name=="post_layernorm"):
                print(f"Layer: {layer_name} - Part {i}, Activation type: {type(act)}")
                act = [f"{diff:.2f}" for diff in act.flatten()] # Flatten the array if it's multi-dimensional
                print(act)
                new_tensor = copy.deepcopy(activation[i])
    else:
      if(layer_name=="post_layernorm"):
        print(f"Layer: {layer_name}, Activation type: {type(activation)}")
        activation = [f"{diff:.2f}" for diff in activation.flatten()] # Flatten the array if it's multi-dimensional
        print(activation)
        new_tensor = copy.deepcopy(activations[layer_name])

Layer: post_layernorm, Activation shape: torch.Size([1, 768])
['0.04', '0.69', '-0.36', '0.43', '1.15', '-1.51', '1.86', '0.30', '-0.06', '0.25', '0.07', '-0.06', '-1.64', '-0.46', '0.68', '1.02', '1.02', '0.77', '-0.75', '-0.64', '-0.47', '0.32', '1.33', '1.51', '-0.90', '1.24', '0.24', '0.59', '-0.07', '1.12', '-1.35', '-0.79', '0.09', '-1.73', '-0.07', '0.45', '0.38', '-0.03', '0.72', '-0.89', '0.41', '1.88', '-1.04', '-0.51', '-0.41', '0.13', '-0.85', '0.98', '0.82', '1.11', '0.83', '0.22', '0.83', '0.27', '0.47', '0.72', '1.41', '0.79', '0.23', '1.15', '-0.42', '-0.41', '0.01', '-0.70', '0.21', '0.37', '-0.35', '0.56', '-0.52', '-0.61', '-0.41', '0.22', '0.69', '0.63', '0.72', '0.06', '0.07', '0.44', '0.40', '0.99', '0.65', '-0.09', '0.30', '-0.36', '0.61', '0.14', '-0.34', '0.69', '0.27', '0.03', '0.75', '0.11', '0.24', '1.07', '-0.92', '0.55', '-0.45', '0.07', '0.24', '0.13', '-0.09', '-0.94', '-0.25', '-0.45', '0.38', '0.11', '-0.31', '0.47', '0.56', '0.11', '1.11', '0.09', '-0

In [ ]:
#print(new_tensor)
#new_tensor[0][0]=0
activation_last = [f"{diff:.2f}" for diff in new_tensor.flatten()] # Flatten the array if it's multi-dimensional
print(activation_last)

['0.04', '0.69', '-0.36', '0.43', '1.15', '-1.51', '1.86', '0.30', '-0.06', '0.25', '0.07', '-0.06', '-1.64', '-0.46', '0.68', '1.02', '1.02', '0.77', '-0.75', '-0.64', '-0.47', '0.32', '1.33', '1.51', '-0.90', '1.24', '0.24', '0.59', '-0.07', '1.12', '-1.35', '-0.79', '0.09', '-1.73', '-0.07', '0.45', '0.38', '-0.03', '0.72', '-0.89', '0.41', '1.88', '-1.04', '-0.51', '-0.41', '0.13', '-0.85', '0.98', '0.82', '1.11', '0.83', '0.22', '0.83', '0.27', '0.47', '0.72', '1.41', '0.79', '0.23', '1.15', '-0.42', '-0.41', '0.01', '-0.70', '0.21', '0.37', '-0.35', '0.56', '-0.52', '-0.61', '-0.41', '0.22', '0.69', '0.63', '0.72', '0.06', '0.07', '0.44', '0.40', '0.99', '0.65', '-0.09', '0.30', '-0.36', '0.61', '0.14', '-0.34', '0.69', '0.27', '0.03', '0.75', '0.11', '0.24', '1.07', '-0.92', '0.55', '-0.45', '0.07', '0.24', '0.13', '-0.09', '-0.94', '-0.25', '-0.45', '0.38', '0.11', '-0.31', '0.47', '0.56', '0.11', '1.11', '0.09', '-0.25', '-0.52', '0.28', '-0.64', '0.74', '-3.87', '-0.30', '1.3

In [ ]:
#red_tensor=copy.deepcopy(new_tensor)
black_tensor=copy.deepcopy(new_tensor)

In [ ]:
activation_red = [f"{diff:.2f}" for diff in red_tensor.flatten()] # Flatten the array if it's multi-dimensional
print(activation_red)

activation_black = [f"{diff:.2f}" for diff in black_tensor.flatten()] # Flatten the array if it's multi-dimensional
print(activation_black)


['0.48', '0.57', '0.60', '-0.27', '1.67', '-0.34', '1.43', '0.78', '0.62', '1.28', '0.43', '-0.74', '-0.59', '-0.06', '0.69', '0.03', '1.48', '0.20', '-0.28', '-0.33', '-1.08', '0.23', '1.94', '1.04', '-0.03', '1.23', '-0.31', '-0.40', '-0.25', '1.25', '-1.38', '-0.82', '0.17', '1.31', '0.66', '0.57', '1.53', '0.66', '0.68', '-1.70', '-0.09', '0.24', '-2.14', '-1.02', '-0.95', '0.50', '0.57', '2.16', '1.18', '1.46', '-0.10', '0.63', '0.92', '0.54', '-1.30', '0.42', '1.30', '1.31', '0.42', '0.87', '-0.79', '-0.83', '0.06', '-0.32', '0.91', '0.08', '0.58', '0.63', '0.43', '0.16', '-0.36', '1.09', '0.11', '0.47', '0.11', '1.12', '0.61', '-0.08', '0.33', '1.45', '0.47', '-1.18', '0.39', '-0.31', '0.91', '-0.28', '0.84', '1.10', '0.06', '0.47', '0.41', '0.58', '1.81', '1.53', '0.32', '1.03', '-0.35', '-0.72', '0.24', '-0.94', '-0.31', '-0.89', '-0.08', '-0.39', '0.09', '0.27', '0.34', '1.85', '0.17', '0.03', '0.74', '-0.59', '0.27', '-0.05', '0.35', '-0.84', '-0.00', '-5.67', '-0.37', '0.93

In [ ]:
diff_pixel= black_tensor - red_tensor
#diff_pixel= red_tensor - black_tensor

activation_diff = [f"{diff:.2f}" for diff in diff_pixel.flatten()] # Flatten the array if it's multi-dimensional
print(activation_diff)



['0.45', '-0.11', '0.95', '-0.70', '0.52', '1.18', '-0.44', '0.48', '0.67', '1.03', '0.35', '-0.68', '1.04', '0.39', '0.02', '-0.99', '0.46', '-0.58', '0.47', '0.31', '-0.61', '-0.09', '0.61', '-0.47', '0.87', '-0.01', '-0.56', '-0.99', '-0.18', '0.13', '-0.03', '-0.04', '0.08', '3.04', '0.73', '0.12', '1.15', '0.70', '-0.04', '-0.81', '-0.49', '-1.64', '-1.09', '-0.51', '-0.54', '0.37', '1.43', '1.18', '0.36', '0.35', '-0.93', '0.42', '0.09', '0.27', '-1.77', '-0.30', '-0.11', '0.52', '0.19', '-0.28', '-0.37', '-0.42', '0.05', '0.38', '0.70', '-0.29', '0.93', '0.07', '0.95', '0.77', '0.05', '0.86', '-0.58', '-0.16', '-0.61', '1.05', '0.54', '-0.52', '-0.08', '0.46', '-0.18', '-1.09', '0.09', '0.05', '0.30', '-0.42', '1.17', '0.41', '-0.21', '0.44', '-0.35', '0.47', '1.57', '0.47', '1.24', '0.47', '0.10', '-0.79', '0.01', '-1.07', '-0.21', '0.05', '0.17', '0.06', '-0.29', '0.16', '0.66', '1.37', '-0.39', '-0.07', '-0.37', '-0.68', '0.52', '0.47', '0.07', '-0.20', '-0.75', '-1.80', '-0.

In [ ]:
magnitude = torch.norm(diff_pixel)

# Calculate the unit vector
unit_vector = diff_pixel / magnitude

print("Unit vector:", unit_vector.shape)
print("New Tensor:", new_tensor.shape)

Unit vector: torch.Size([1, 768])
New Tensor: torch.Size([1, 768])


In [ ]:
new_tensor =copy.deepcopy(red_tensor + unit_vector)
print(new_tensor.shape)

torch.Size([1, 768])


In [ ]:
update_tensor = [f"{diff:.2f}" for diff in new_tensor.flatten()] # Flatten the array if it's multi-dimensional
print(update_tensor)


['0.51', '0.57', '0.64', '-0.30', '1.69', '-0.28', '1.41', '0.80', '0.65', '1.33', '0.44', '-0.77', '-0.54', '-0.05', '0.69', '-0.02', '1.50', '0.17', '-0.26', '-0.32', '-1.11', '0.22', '1.97', '1.02', '0.01', '1.23', '-0.34', '-0.45', '-0.25', '1.26', '-1.38', '-0.82', '0.18', '1.45', '0.70', '0.58', '1.59', '0.70', '0.68', '-1.73', '-0.11', '0.16', '-2.19', '-1.04', '-0.97', '0.52', '0.64', '2.21', '1.20', '1.48', '-0.14', '0.65', '0.93', '0.55', '-1.38', '0.41', '1.29', '1.33', '0.43', '0.85', '-0.81', '-0.85', '0.07', '-0.30', '0.94', '0.06', '0.62', '0.63', '0.47', '0.19', '-0.36', '1.13', '0.08', '0.46', '0.08', '1.17', '0.64', '-0.11', '0.32', '1.47', '0.46', '-1.23', '0.39', '-0.31', '0.93', '-0.30', '0.89', '1.12', '0.05', '0.49', '0.39', '0.60', '1.88', '1.56', '0.38', '1.05', '-0.35', '-0.76', '0.25', '-0.99', '-0.32', '-0.88', '-0.07', '-0.38', '0.07', '0.28', '0.37', '1.91', '0.15', '0.03', '0.72', '-0.62', '0.30', '-0.03', '0.35', '-0.85', '-0.04', '-5.75', '-0.37', '0.91

In [ ]:
print(new_tensor.shape)
print(red_tensor.shape)
print(black_tensor.shape)

torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])


#Update Activation

In [ ]:
# Access the 'post_layernorm' layer and update its parameters
for name, module in model.vision_model.named_modules():
    if name == "post_layernorm":
        for param_name, param in module.named_parameters():
            print(f"Parameter name: {param_name}")
            #print(f"Original parameter value:\n{param}")
            # Ensure the shape of new_tensor matches the shape of the original parameter
            if new_tensor[0].shape == param.shape:
                with torch.no_grad():
                    param.copy_(new_tensor[0])
            else:
                print(f"Shape mismatch for {param_name}: expected {param.shape}, got {new_tensor.shape}")


Parameter name: weight
Parameter name: bias


#Print updated tensor

In [ ]:
import copy

# Define a hook to capture the activations
activations = {}

# image = Image.open("black.png")
# inputs = processor(images=image, return_tensors="pt")

# Register hooks for all layers
for name, module in model.vision_model.named_modules():
    module.register_forward_hook(get_activation(name))

# Run the image through the model
with torch.no_grad():
    model.get_image_features(**inputs)


for layer_name, activation in activations.items():
    if isinstance(activation, torch.Tensor):
      if(layer_name=="post_layernorm"):
        print(f"Layer: {layer_name}, Activation shape: {activation.shape}")
        activation = [f"{diff:.2f}" for diff in activation.flatten()] # Flatten the array if it's multi-dimensional
        print(activation)


    elif isinstance(activation, tuple):
        for i, act in enumerate(activation):
            if isinstance(act, torch.Tensor):
              if(layer_name=="post_layernorm"):
                print(f"Layer: {layer_name} - Part {i}, Activation shape: {act.shape}")
                act = [f"{diff:.2f}" for diff in act.flatten()] # Flatten the array if it's multi-dimensional
                print(act)

            else:
              if(layer_name=="post_layernorm"):
                print(f"Layer: {layer_name} - Part {i}, Activation type: {type(act)}")
                act = [f"{diff:.2f}" for diff in act.flatten()] # Flatten the array if it's multi-dimensional
                print(act)

    else:
      if(layer_name=="post_layernorm"):
        print(f"Layer: {layer_name}, Activation type: {type(activation)}")
        activation = [f"{diff:.2f}" for diff in activation.flatten()] # Flatten the array if it's multi-dimensional
        print(activation)


Layer: post_layernorm, Activation shape: torch.Size([1, 768])
['0.44', '1.23', '0.30', '-0.17', '3.94', '0.69', '4.10', '0.54', '0.46', '1.28', '0.61', '-0.55', '0.57', '-0.01', '1.32', '-0.04', '2.78', '0.26', '0.12', '0.01', '-0.05', '0.20', '2.86', '3.10', '-0.01', '2.37', '-0.28', '-0.70', '-0.26', '2.27', '0.15', '0.23', '0.21', '-0.41', '0.57', '0.72', '1.74', '0.87', '1.09', '0.17', '-0.26', '0.42', '1.13', '-0.41', '0.42', '0.09', '-0.30', '3.80', '2.00', '2.87', '-0.25', '0.70', '2.25', '0.89', '-1.95', '0.60', '2.44', '2.06', '0.45', '1.51', '0.19', '-0.27', '0.06', '0.06', '0.85', '0.05', '0.23', '0.92', '-0.02', '-0.01', '-0.24', '1.33', '0.13', '0.61', '0.10', '0.85', '0.47', '-0.10', '0.55', '2.94', '0.87', '-0.36', '0.48', '-0.10', '1.24', '-0.27', '0.11', '2.19', '0.05', '0.52', '0.71', '0.52', '1.80', '3.33', '-0.13', '1.78', '-0.16', '-0.45', '0.27', '-0.96', '-0.30', '0.78', '-0.05', '0.09', '0.10', '0.23', '-0.04', '2.42', '0.24', '0.03', '1.63', '-0.90', '0.09', '-

#similarity

In [ ]:
# Encode the text
image = Image.open("green.png")
text = ["A picture with red background"]
inputs = processor(text=text, images=image, return_tensors="pt", padding=True)
image_features=None

#after = model.get_text_features(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask)

# Get image and text features
with torch.no_grad():
    # Pass only the image data to get_image_features
    image_features = model.get_image_features(pixel_values=inputs.pixel_values)
    # Pass only the text data to get_text_features
    text_features = model.get_text_features(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask)


# Print features
print("Image features:", image_features.shape)
print("Text features:", text_features.shape)
print(max(image_features[0]))
count=0

# for i in range(512):
#     if(image_features[0][i] < 5):
#          #image_features[0][i]=0
#          count+=1

# Compute cosine similarity between image and text features
cosine_similarity = torch.nn.functional.cosine_similarity(image_features, text_features, dim=1)
print("Cosine similarity:", cosine_similarity," --",count)

Image features: torch.Size([1, 512])
Text features: torch.Size([1, 512])
tensor(3.3571)
Cosine similarity: tensor([0.3179])  -- 0
